# Tracking Models to Run Inferences on the MOT-16 dataset

Multi-Object Tracking (MOT) describes the task of identifying all objects in an image and following them over a sequence of frames. 

A wide range of Deep Learning-based MOT methods has been developed for human detection, but not for sparse/crowded space detection.

### 1. Install Packages and Setup Environment

In [1]:
# Get the FairMOT Model
! git clone https://github.com/ifzhang/FairMOT     # the FairMot scripts
! git clone https://github.com/CharlesShang/DCNv2  # package for using DLA-34 model, the backbone neural network of FairMOT

fatal: destination path 'FairMOT' already exists and is not an empty directory.
fatal: destination path 'DCNv2' already exists and is not an empty directory.


#### Ensure CUDA is enabled

In [2]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


#### Ensure the Python version is 3.8

In [3]:
! python3 --version

Python 3.7.10


#### Install PyTorch

In [4]:
! pip install torch==1.4.0 torchvision==0.5.0

#### Install CONDA

If CONDA is already installed, skip this section.

In [5]:
! conda --version

conda 4.12.0


Install Conda which is required for Fair MOT's requirements.

In [6]:
! pip install -q condacolab

import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


#### Build DCNv2 - Deformable Convolutional Networks v2 with Pytorch

Ensure GPU is enabled before running build. 

Debugging link: https://github.com/CharlesShang/DCNv2/issues/98#issuecomment-939261953

Remove the build folder if it exists

In [7]:
! rm -rf /content/build

Build DCNv2

In [8]:
! python ./DCNv2/setup.py build develop

running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/DCNv2
copying DCNv2/setup.py -> build/lib.linux-x86_64-3.7/DCNv2
copying DCNv2/testcpu.py -> build/lib.linux-x86_64-3.7/DCNv2
copying DCNv2/__init__.py -> build/lib.linux-x86_64-3.7/DCNv2
copying DCNv2/testcuda.py -> build/lib.linux-x86_64-3.7/DCNv2
copying DCNv2/dcn_v2.py -> build/lib.linux-x86_64-3.7/DCNv2
running build_ext
building '_ext' extension
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/content
creating build/temp.linux-x86_64-3.7/content/DCNv2
creating build/temp.linux-x86_64-3.7/content/DCNv2/src
creating build/temp.linux-x86_64-3.7/content/DCNv2/src/cpu
creating build/temp.linux-x86_64-3.7/content/DCNv2/src/cuda
g++ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -DWITH_CUDA -I/content/DCNv2/src -I/usr/local/lib/python3.7/site-packages/torch/include -I/usr/local/lib/python3.7/site-packages/torch/include/tor

Run test CUDA.  

For the *UserWarning: At least one of the inputs that requires gradient is not of double precision floating point. This check will likely fail if all the inputs are not of double precision floating point. 'At least one of the inputs that requires gradient '* error:

* These are known issues on DCNv2 and are related to shortcomings related to grad-check, it is working fine for me (Even the Jacobian mismatch error is occuring).

Source: https://github.com/Mukosame/Zooming-Slow-Mo-CVPR-2020/issues/11#issuecomment-611352001

In [9]:
! python /content/DCNv2/testcuda.py

error in modulated_deformable_im2col_cuda: no kernel image is available for execution on the device
error in modulated_deformable_col2im_coord_cuda: no kernel image is available for execution on the device
error in modulated_deformable_col2im_cuda: no kernel image is available for execution on the device
error in modulated_deformable_im2col_cuda: no kernel image is available for execution on the device
Traceback (most recent call last):
  File "/content/DCNv2/testcuda.py", line 255, in <module>
    example_dconv()
  File "/content/DCNv2/testcuda.py", line 179, in example_dconv
    error.backward()
  File "/usr/local/lib/python3.7/site-packages/torch/tensor.py", line 118, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/usr/local/lib/python3.7/site-packages/torch/autograd/__init__.py", line 93, in backward
    allow_unreachable=True)  # allow_unreachable flag
  File "/usr/local/lib/python3.7/site-packages/torch/autograd/function.py", line 77, 

#### Install FairMOT requirements

In [10]:
! pip install -r /content/FairMOT/requirements.txt
! conda install pytorch==1.2.0 torchvision==0.4.0 cudatoolkit=10.0 -c pytorch -y
! conda install ffmpeg -y

  Using cached yacs-0.1.8-py3-none-any.whl (14 kB)
  Using cached PyYAML-6.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (596 kB)
  Using cached cython_bbox-0.1.3.tar.gz (41 kB)
  Using cached cython_bbox-0.1.2.tar.gz (40 kB)
ERROR: Could not find a version that satisfies the requirement cython-bbox
ERROR: No matching distribution found for cython-bbox
Solving environment: - \ | / - \ | / - \ | / - WARNING conda.core.solve:_add_specs(649): pinned spec cudatoolkit=11.1 conflicts with explicit specs.  Overriding pinned spec.
\ | / - \ | / - \ | done

# All requested packages already installed.

Solving environment: - \ | / - \ | / - \ | / - \ | / - done

# All requested packages already installed.



#### Install packages for the DLA-34 Model

If these packages were missing, make sure to restart your runtime after installing.

In [11]:
! pip install boto3
! pip install opencv-python
! pip install sagemaker

#### Download pre-trained model weights of the DLA-34 backbone model

In [13]:
import boto3
import cv2
from PIL import Image
import sagemaker

sess = sagemaker.Session()
s3 = boto3.client('s3')

# S3 bucket name and directories for the pre-trained DLA34 model
bucket = 'pedestrian-tracker'  
s3key_model = 'raw-pretrained-model'
fname = 'fairmot_dla34.pth'
s3.download_file(bucket, s3key + '/' + fname,  '/content/FairMOT/model/' + fname)


ValueError: ignored